## Data loading

In [ ]:
import h5py
import numpy as np

from pathlib import Path


In [6]:
data_dir = Path("./data/")
assert data_dir.is_dir()
intra_dir = data_dir / "Intra"
cross_dir = data_dir / "Cross"
intra_train_glob = list((intra_dir / "train").glob("*.h5"))
intra_test_glob = list((intra_dir / "test").glob("*.h5"))
len(intra_train_glob)

32

In [7]:
def load_labels(path: Path) -> np.ndarray:
    *task, subject_identifier, chunk = path.stem.split("_")
    if "rest" in task:
        y = 0
    elif 'math' in task:
        y = 1
    elif 'working' in task:
        y = 2
    elif 'motor' in task:
        y = 3
    else:
        assert False, 'unknown task'
    return np.array([y, int(subject_identifier), int(chunk)])

In [8]:
def load_h5(path: Path) -> np.ndarray:
    with h5py.File(path) as f:
        keys = f.keys()
        assert len(keys) == 1, f"Only one key per file, right? {intra_train_glob[0]}"
        matrix = f.get(next(iter(keys)))[()]
    return matrix


intra_train_X = np.stack(list(map(load_h5, intra_train_glob)))
intra_train_labels = np.stack(list(map(load_labels, intra_train_glob)))
intra_train_X.shape, intra_train_labels.shape

((32, 248, 35624), (32, 3))

In [9]:
intra_test_X = np.stack(list(map(load_h5, intra_test_glob)))
intra_test_labels = np.stack(list(map(load_labels, intra_test_glob)))
intra_test_X.shape, intra_test_labels.shape

((8, 248, 35624), (8, 3))

## Data preprocessing

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


# Downsample data to 125Hz
downsampled_X = torch.nn.functional.interpolate(torch.Tensor(X), scale_factor=125/2034, mode='linear')

# Time-wise Z-score normalization
scaler = torch.nn.functional.normalize 
scaled_X = torch.Tensor([scaler(x, dim=1) for x in downsampled_X.numpy()])


# Apply spatial de-mixing (You need to implement your own spatial de-mixing)
# This could involve reshaping the data, applying spatial filters, etc.
pass

# Convert data to PyTorch Dataset
dataset = TensorDataset(torch.Tensor(scaled_X), torch.LongTensor(y))

# K-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## VAR-CNN Architecture
implemented from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6609925/

In [ ]:
# Model Architecture
class YourModel(nn.Module):
    def __init__(self, k):
        super(YourModel, self).__init__()
        # Define your layers here

    def forward(self, x):
        # Define the forward pass
        return x

# Instantiate your model
model = YourModel(k=...)  # Specify the value of k


## Training

In [ ]:
# Training
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Training loop and cross-validation
for train_index, val_index in kf.split(scaled_X, y):
    train_data, val_data = dataset[train_index], dataset[val_index]
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=100, shuffle=False)

    # Implement the training loop with early stopping and l1 regularization
    # You need to keep track of the best model based on validation performance
    # Train for 100 iterations with early stopping and l1 regularization

# After training, evaluate on your test set using the best model
# Implement your evaluation code here
